In [213]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

In [233]:
#Choose either "utility_side" or "total_cost." Utility-side is incentive costs + nonincentive costs. Total cost is non-incentive costs + measure cost.
cost_type = 'total_cost'

In [234]:
res_ee_costs = pd.read_csv('data/cmua_ladwp_residential_EEcosts.csv')
comm_ee_costs = pd.read_csv('data/cmua_ladwp_C&I_EEcosts.csv')

In [235]:
resource_projected_costs = pd.read_csv('data/resource_projected_costs.csv')

In [236]:
ee_savings_shapes = pd.read_csv('data/gen_profiles.csv')

In [237]:
map_ee_comm_loadshapes_to_costs = {
    'DaylightingEff':'Interior Lighting', 
    'IndoorLightingEff':'Interior Lighting',
    'OutdoorLighting': 'Exterior Lighting',
    'Cooling':'Cooling', 
    'EconomizerEff':'Cooling',
    'Heating':'Heating',
    'Ventilation':'Ventilation', 
    'WaterHeating':'Hot Water', 
    'OfficeEquipment':'Plug Loads',
    'Refrigeration':'Refrigeration', 
    'Cooking':'Cooking', 
}

map_ee_res_loadshapes_to_costs = {
    'Dryer':'Appliance', 
    'Refrigerator':'Appliance',
    'Washer':'Appliance',
    'CoolingEff':'Cooling',
    'Heating':'HVAC', 
    'ResLightingEff':'Lighting', 
    'WaterHeating':'Water Heating',
}

In [238]:
ee_measures = resource_projected_costs[resource_projected_costs['resource'].str.startswith('FCZ7')]['resource'].unique()

In [239]:
for measure in ee_measures:
    end_use = np.nan

    annual_kwh_savings = ee_savings_shapes[measure].sum()

    if 'Commercial' in measure:
        enduse_map = map_ee_comm_loadshapes_to_costs
        for key in enduse_map:
            if key in measure:
                end_use = map_ee_comm_loadshapes_to_costs[key]
    
        costs = comm_ee_costs[comm_ee_costs['end_use']== end_use]

    elif 'Residential' in measure:
        enduse_map = map_ee_res_loadshapes_to_costs
        for key in enduse_map:
            if key in measure:
                end_use = map_ee_res_loadshapes_to_costs[key]

        costs = res_ee_costs[res_ee_costs['end_use']== end_use]
    
    if not costs.empty:
        
        for year in costs.columns[2:]:
            if cost_type == 'total_cost':
                nonincentive_costs = costs[costs['cost_type']=='nonincentive_dollars_per_first_year_kwh_saved']
                nonincentive_cost_year = float(nonincentive_costs[year].item())

                measure_costs = costs[costs['cost_type']=='measure_cost_per_first_year_kwh_saved']
                measure_cost_year = float(measure_costs[year].item())

                cost_per_kwh = nonincentive_cost_year + measure_cost_year

            if cost_type == 'utility_side':
                nonincentive_costs = costs[costs['cost_type']=='nonincentive_dollars_per_first_year_kwh_saved']
                nonincentive_cost_year = float(nonincentive_costs[year].item())

                incentive_costs = costs[costs['cost_type']=='incentive_dollars_per_first_year_kwh_saved']
                incentive_cost_year = float(incentive_costs[year].item())

                cost_per_kwh = nonincentive_cost_year + incentive_cost_year

            cost_per_kw = cost_per_kwh * annual_kwh_savings

            measure_inds = resource_projected_costs['resource']==measure
            cost_type_inds = resource_projected_costs['cost_type']== 'capex_per_kw'

            resource_projected_costs.loc[measure_inds & cost_type_inds, year] = cost_per_kw
            
            weighted_avg_eul = costs[costs['cost_type']=='weighted_avg_eul']
            weighted_avg_eul_year = float(weighted_avg_eul[year].item())
            
            eul_inds = resource_projected_costs['cost_type']== 'weighted_avg_eul'
            resource_projected_costs.loc[measure_inds & eul_inds, year] = weighted_avg_eul_year


In [240]:
resource_projected_costs

,resource,cost_type,cost_decline_assumption,2018,2019,2020,2021,2022,2023,2024,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,gas_ct,capex_per_kw,advanced,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
1,gas_ct,capex_per_kw,moderate,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
2,gas_ct,capex_per_kw,conservative,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
3,gas_cc,capex_per_kw,advanced,1088.327739,1077.13417,1065.940601,1054.747032,1043.553463,1032.359894,1018.169642,...,946.331657,943.043161,940.402818,937.718126,935.560219,931.695997,929.062208,926.389946,923.698379,913.052378
4,gas_cc,capex_per_kw,moderate,1088.327739,1077.13417,1065.940601,1054.747032,1043.553463,1032.359894,1018.169642,...,946.331657,943.043161,940.402818,937.718126,935.560219,931.695997,929.062208,926.389946,923.698379,913.052378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,FCZ7.Residential.SINGLEFAMILY.Refrigerator,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,9.700000,9.700000,9.700000,...,10.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,FCZ7.Residential.SINGLEFAMILY.ResLightingEff,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,18.400000,18.500000,19.000000,...,16.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
905,FCZ7.Residential.SINGLEFAMILY.ResOccsensorEff,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906,FCZ7.Residential.SINGLEFAMILY.Washer,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,9.700000,9.700000,9.700000,...,10.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
measure_nans = resource_projected_costs[resource_projected_costs['2022'].isna()]['resource'].unique()

In [242]:
resource_projected_costs = resource_projected_costs[resource_projected_costs['resource'].isin(measure_nans)==False]

In [243]:
ee_resource_potential = pd.read_csv('data/ee_resource_potential_2020_2030.csv')

In [244]:
ee_measures_w_resource_potential = ee_resource_potential['resource'].unique().tolist()

In [245]:
ee_measures_full = resource_projected_costs[resource_projected_costs['resource'].str.startswith('FCZ7')]['resource'].unique().tolist()

In [246]:
missing_measures = list(set(ee_measures_full).difference(ee_measures_w_resource_potential))

In [247]:
resource_projected_costs = resource_projected_costs[resource_projected_costs['resource'].isin(missing_measures)==False]

In [248]:
resource_projected_costs

,resource,cost_type,cost_decline_assumption,2018,2019,2020,2021,2022,2023,2024,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,gas_ct,capex_per_kw,advanced,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
1,gas_ct,capex_per_kw,moderate,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
2,gas_ct,capex_per_kw,conservative,982.962461,978.28410,973.605739,968.927377,964.249016,953.168284,936.035607,...,863.759109,860.870545,858.573699,856.236349,854.380081,850.965521,848.674711,846.348864,844.005297,834.392535
3,gas_cc,capex_per_kw,advanced,1088.327739,1077.13417,1065.940601,1054.747032,1043.553463,1032.359894,1018.169642,...,946.331657,943.043161,940.402818,937.718126,935.560219,931.695997,929.062208,926.389946,923.698379,913.052378
4,gas_cc,capex_per_kw,moderate,1088.327739,1077.13417,1065.940601,1054.747032,1043.553463,1032.359894,1018.169642,...,946.331657,943.043161,940.402818,937.718126,935.560219,931.695997,929.062208,926.389946,923.698379,913.052378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,FCZ7.Residential.SINGLEFAMILY.Dryer,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,9.700000,9.700000,9.700000,...,10.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,FCZ7.Residential.SINGLEFAMILY.Heating,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,7.200000,7.200000,7.100000,...,10.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903,FCZ7.Residential.SINGLEFAMILY.Refrigerator,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,9.700000,9.700000,9.700000,...,10.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,FCZ7.Residential.SINGLEFAMILY.ResLightingEff,weighted_avg_eul,moderate,NaN,NaN,NaN,NaN,18.400000,18.500000,19.000000,...,16.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
resource_projected_costs.to_csv('data/resource_projected_costs_ee_totalcosts.csv')